# Introduction to time-series analysis of HyP3 product with MintPy

## Introduction
This notebook shows you how to do time-series analysis against HyP3 product with MintPy. This tutorial assumes that you have already successfully installed the MintPy and created the conda environment.
Detail installation info is refferered at https://github.com/insarlab/MintPy.

## Run the application

Before running the program, some package need to be installed and imported.

In [ ]:
!conda install -y -c conda-forge hyp3_sdk ipywidgets

In [ ]:
import os
import re
import glob
import zipfile
import numpy as np
import hyp3_sdk as sdk

Set project name, output directory and change to the output directory

In [ ]:
projectname = input("project name:") or "case5"
outdir = input("home directory of the project") or "./tutorial"
os.system("mkdir -p {}".format(outdir))
print(projectname)
print(outdir)
os.chdir(outdir)

### Download the hyp3 data

The granule_list file includes the RTC file names that users wants to use them to produce the interferograms for the time-series analysis purpose. Users usually prepare this file with ASF vertex (https://serach.asf.alaska.edu). In this example, we assume users have the granule_list in their hands. The content of the granule_list are:

User autentication

In [ ]:
hyp3 = sdk.HyP3(prompt = True)

Submit the jobs with hyp3-sdk and download the files 

In [ ]:
reference_granule = 'S1A_IW_SLC__1SSV_20160112T205113_20160112T205140_009466_00DBA8_C430'

secondary_granules = [
    'S1A_IW_SLC__1SSV_20160205T205112_20160205T205139_009816_00E5D1_997E',
    'S1A_IW_SLC__1SSV_20160229T205112_20160229T205139_010166_00EFF4_357E',
    'S1A_IW_SLC__1SSV_20160324T205112_20160324T205139_010516_00F9E8_FA1C',
    'S1A_IW_SLC__1SSV_20160417T205113_20160417T205140_010866_010443_7429',
    'S1A_IW_SLC__1SSV_20160511T205115_20160511T205142_011216_010F42_BD06',
    'S1A_IW_SLC__1SSV_20160604T205107_20160604T205135_011566_011A9C_EDD4',
    'S1A_IW_SLC__1SSV_20160628T205108_20160628T205136_011916_01259C_EE11'
]

In [ ]:
insar_jobs = sdk.Batch()

for secondary_granule in secondary_granules:
    insar_jobs += hyp3.submit_insar_job(reference_granule, secondary_granule,
                                        name=projectname, include_inc_map=True,
                                        include_dem=True)

In [ ]:
insar_jobs = hyp3.watch(insar_jobs)

In [ ]:
zip_files = insar_jobs.download_files()

for zip_file in zip_files:
    with zipfile.ZipFile(zip_file) as zip_ref:
        zip_ref.extractall(outdir)

Downloaded ifm files are here

In [ ]:
! tree {outdir}

### Organize the data into the structure that MintPy requests

#### Clip the geotiff files in the current directory and its subdirectories

In [ ]:
from osgeo import gdal

pattern = os.path.join(outdir, '**/*.tif')
files = glob.glob(pattern)

# determine the smallest area covered by all input files
corners = [gdal.Info(f, format='json')['cornerCoordinates'] for f in files]
ulx = max(corner['upperLeft'][0] for corner in corners)
uly = min(corner['upperLeft'][1] for corner in corners)
lrx = min(corner['lowerRight'][0] for corner in corners)
lry = max(corner['lowerRight'][1] for corner in corners)

# subset all input files to these common coordinates
for f in files:
    output_filename = f.replace('.tif', '_clip.tif')
    gdal.Translate(destName=output_filename, srcDS=f, projWin=[ulx, uly, lrx, lry])

Prepared data structure are:

In [ ]:
! tree {outdir}

### Configure the custom cfg file in the outdir

Show the cunstom cfg file  

In [ ]:
from pathlib import Path

config = f'''
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {outdir}/*/*unw_phase_clip.tif
mintpy.load.corFile          = {outdir}/*/*corr_clip.tif
##---------geometry datasets:
mintpy.load.demFile          = {outdir}/*/*dem_clip.tif
mintpy.load.incAngleFile     = {outdir}/*/*inc_map_clip.tif
'''
config_file = f'{projectname}.cfg'
Path(config_file).write_text(config)

In [ ]:
f=open(config_file)
lines = f.readlines()
print("\n".join(lines))
f.close()

### Run Time-series Analysis application


In [ ]:
from mintpy import smallbaselineApp
smallbaselineApp.main([config_file])

## Display the analysis results

There are a few scripts used to display the analysis results. There are in the MINTPY_HOME/mintpy. Here we show two majoy disaply scripts.

### Disaply velocity of displacement


In [ ]:
%matplotlib notebook
from mintpy import view
view.main(['velocity.h5'])

### Display the time-series of displacement

In [ ]:
%matplotlib notebook
from mintpy import tsview
tsview.main(['timeseries.h5'])